In [1]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!unzip '/content/archive.zip'

Archive:  /content/archive.zip
replace Alzheimer_s Dataset/test/MildDemented/26 (19).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
imageSize = [224, 224]

trainPath = r"/content/Alzheimer_s Dataset/train"

testPath = r"/content/Alzheimer_s Dataset/test"


In [4]:
vgg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [5]:
for layer in vgg.layers:
  layer.trainable = False

In [6]:
x = Flatten()(vgg.output)

In [7]:
prediction = Dense(4, activation='softmax')(x)

In [8]:
model = Model(inputs=vgg.input, outputs=prediction)

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [15]:
import sys
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=50,
  validation_steps=171//10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
50/50 [==============================] - 356s 7s/step - loss: 0.8996 - accuracy: 0.6720 - val_loss: 1.6080 - val_accuracy: 0.5941
Epoch 2/25
50/50 [==============================] - 358s 7s/step - loss: 0.6291 - accuracy: 0.7500 - val_loss: 0.9544 - val_accuracy: 0.6118
Epoch 3/25
50/50 [==============================] - 357s 7s/step - loss: 0.6558 - accuracy: 0.7400 - val_loss: 1.3245 - val_accuracy: 0.5824
Epoch 4/25
50/50 [==============================] - 354s 7s/step - loss: 0.8963 - accuracy: 0.6904 - val_loss: 0.9884 - val_accuracy: 0.5765
Epoch 5/25
50/50 [==============================] - 355s 7s/step - loss: 0.6382 - accuracy: 0.7560 - val_loss: 1.1198 - val_accuracy: 0.5706
Epoch 6/25
50/50 [==============================] - 360s 7s/step - loss: 1.1938 - accuracy: 0.6000 - val_loss: 1.3052 - val_accuracy: 0.6000
Epoch 7/25
50/50 [==============================] - 353s 7s/step - loss: 0.7774 - accuracy: 0.6960 - val_loss: 1.8641 - val_accuracy: 0.5471
Epoch 8/25
50

In [16]:
model.save('adp.h5')

In [20]:
!tar -zcvf adp.tgz adp.h5

adp.h5


In [33]:
!pip install watson-machine-learning-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
!pip install ibm_watson_machine_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"6CgMGPrwd4fg3LftcNFW5nif2dEfWE_5TyZV13mpslL8"
}

client = APIClient(wml_credentials)
client

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [39]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/d2155da9f8314ed5b9f1d91f55819a9b:3001af94-5c57-4347-9715-73a9741163ef::',
      'guid': '3001af94-5c57-4347-9715-73a9741163ef',
      'name': 'Watson Machine Learning-ps',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'adp',
    'scope': {'bss_account_id': 'd2155da9f8314ed5b9f1d91f55819a9b'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '7379bcaa-dde9-418b-b436-ac570ad2317c',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': 'd834c4f4b4fd407bb054f5ad2a004f09',
        'api_key': 'Uqc-9o7iPv8xfG9bfzlEKcWMZyymiomDIDOtf_eOvNuI',
        'secret_access_key': '61673674ab642bb36f9c9e88798c40ddcd80302b6e96529f',
        'service_id': 'ServiceId-6b3f55bf-c5e1-42f9-b571-ba67200cc0d3'},
       'editor': {'access_key_id': '1e5e92182fc043368d66b9d6e8e3bc86',
        'api_key': '0iZXD

In [40]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ----  ------------------------
ID                                    NAME  CREATED
14f2af36-b926-4835-9837-fe62b001a2b4  adp   2022-11-11T17:15:23.479Z
------------------------------------  ----  ------------------------


In [41]:
space_uid = "14f2af36-b926-4835-9837-fe62b001a2b4"
space_uid

'14f2af36-b926-4835-9837-fe62b001a2b4'

In [42]:
client.set.default_space(space_uid)

'SUCCESS'

In [43]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [44]:
software_space_uid = client.software_specifications.get_uid_by_name('runtime-22.1-py3.9')
software_space_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [52]:
import tensorflow
tensorflow.__version__

'2.9.2'

In [55]:
model_details = client.repository.store_model(model="adp.tgz", meta_props={
    client.repository.ModelMetaNames.NAME:"cnn",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [56]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-11T18:08:53.886Z',
  'id': '0bd3519d-bf10-4e1d-bc54-34e5ba73b1fa',
  'modified_at': '2022-11-11T18:08:58.637Z',
  'name': 'cnn',
  'owner': 'IBMid-6610044C0E',
  'resource_key': '608932b1-f5b6-4e4c-b285-c109f2c3d564',
  'space_id': '14f2af36-b926-4835-9837-fe62b001a2b4'},
 'system': {'warnings': []}}

In [57]:
model_id = client.repository.get_model_id(model_details)
model_id

'0bd3519d-bf10-4e1d-bc54-34e5ba73b1fa'

In [58]:
client.repository.download(model_id,'cnn_adp.tgz')

Successfully saved model content to file: 'cnn_adp.tgz'


'/content/cnn_adp.tgz'